In [2]:
!pip install gputil
!pip install psutil
!pip install humanize

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=0682bc47ec541d381ee5fa9e67dbd17fe1f4bd3f6e52a6695b23ab3bd7b3e8aa
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [3]:
# Import packages
import os,sys,humanize,psutil,GPUtil

# Define function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))
    
# Execute function
mem_report()

CPU RAM Free: 12.8 GB
GPU 0 ... Mem Free: 16280MB / 16280MB | Utilization   0%


In [0]:
!pip install numpy==1.17

In [12]:
# Sample code from the TorchVision 0.3 Object Detection Finetuning Tutorial
# http://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
import json
import os
import sys
sys.path.append('/content/drive/Shared drives/Projet ROBOT/segmentation')
import numpy as np
import torch
from PIL import Image,ImageDraw
import warnings
warnings.filterwarnings('ignore')


import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.transforms import functional as F

from engine import train_one_epoch, evaluate
import utils
import transforms as T

#torch.backends.cudnn.enabled = False

def seg2mask(w,h,seg):
    img = Image.new('L',(w,h),0)
    ImageDraw.Draw(img).polygon(seg,outline=1,fill=1)
    img = np.array(img)
    return img

def make_target(annotations,idx,w,h):

    target = {'boxes':[],
              'labels':[],
              'image_id':idx,
              'area':[],
              'iscrowd':[],
              'masks':[]}
    for i in range(len(annotations)):
        num_class = 0
        if int(annotations[i]['image_id']) > idx:
            break
        if int(annotations[i]['image_id']) == idx:
            num_class += 1          
            target['labels'].append(annotations[i]['category_id'])
            target['area'].append(annotations[i]['area'])
            target['iscrowd'].append(annotations[i]['iscrowd'])
            mask = seg2mask(h,w,annotations[i]['segmentation'])
            pos = np.where(mask)
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            target['masks'].append(mask)
            target['boxes'].append([xmin, ymin, xmax, ymax])
    target['boxes'] = torch.tensor(target['boxes'],dtype=torch.float32)
    target['labels'] = torch.tensor(target['labels'],dtype=torch.int64)
    target['image_id'] = torch.tensor(target['image_id'],dtype=torch.int64)
    target['area'] = torch.tensor(target['area'],dtype=torch.float64)
    target['iscrowd'] = torch.tensor(target['iscrowd'],dtype=torch.uint8)
    target['masks'] = torch.tensor(target['masks'],dtype=torch.uint8)
    return target

class ValDataset(object):
    def __init__(self, transforms):
        self.root = '/content/drive/Shared drives/Projet ROBOT/segmentation/images2/'
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        """
        Decompresser le JSON
        """
        json_file = open('/content/drive/Shared drives/Projet ROBOT/segmentation/images2/annotations.json','r')
        json_data = json_file.readline()
        json_file.close()
        json_data = json.loads(json_data)
        self.annotations = json_data['annotations']
        self.imgs = json_data['images']


    def __getitem__(self, idx):
        # load images ad masks
        if idx == 23:
          idx = 22
        img_name = self.imgs[idx]['file_name']
        img_path = self.root+img_name
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        target = make_target(self.annotations,idx,img.height,img.width)

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

def read_annotations(idx,path,h,w):
  boxes = []
  labels = []
  areas =[]
  f = open(path)
  for line in f.readlines():
    line = line.split(' ')
    try:
      labels.append(int(line[0]))
    except:
      labels.append(5)
    b = [float(line[1])*w,float(line[2])*h,float(line[3])*w,float(line[4])*h]
    if b[0]>b[2]:
      b[0] , b[2] = b[2] , b[0]
    if b[1]>b[3]:
      b[1],b[3]= b[3],b[1]
    boxes.append(np.array(b))
    areas.append((b[2]-b[0])*(b[3]-b[1]))
  target = {'labels':torch.tensor(labels,dtype=torch.int64),
            'boxes':torch.tensor(boxes,dtype=torch.float32),
            'image_id':torch.tensor([idx],dtype=torch.int64),
            'area': torch.tensor(areas,dtype=torch.float64),
            'iscrowd':torch.zeros((len(areas),), dtype=torch.int64)}
  return target




class TrainDataset(object):
    def __init__(self, transforms):
        self.root = '/content/drive/Shared drives/Projet ROBOT/segmentation/data_federico/'
        self.transforms = transforms
        self.listing_imgs = [i for i in os.listdir(self.root) if '.png' in i]
        self.listing_annotations = [i for i in os.listdir(self.root) if '.txt' in i]
        # load all image files, sorting them to
        # ensure that they are aligned

    def __getitem__(self, idx):
        # load images ad masks
        img_name = self.listing_imgs[idx]
        annotation_name = self.listing_annotations[idx]
        img_path = self.root+img_name
        annotation_path = self.root+annotation_name
        img = Image.open(img_path).convert("RGB")
        img = img.resize((img.size[0]//2,img.size[1]//2),Image.ANTIALIAS)
        if self.transforms is not None:
            img = F.to_tensor(img)
        target = read_annotations(idx,annotation_path,img.shape[1],img.shape[2])
        
        return img, target

    def __len__(self):
        return len(self.listing_imgs)

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

"""
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, progress=True, pretrained_backbone=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model
"""
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)


def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    #device = torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = 10
    # use our dataset and defined transformations
    dataset = ValDataset(get_transform(train=True))
    dataset_test = ValDataset(get_transform(train=False))
    #dataset_test = ValDataset(get_transform(train=False))
    # split the dataset in train and test set

    indices = torch.randperm(len(dataset)).tolist()
    dataset = torch.utils.data.Subset(dataset_test, indices[:-10])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-10:])
    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=4, shuffle=True, num_workers=0,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=0,
        collate_fn=utils.collate_fn)

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad] #model.parameters()
    optimizer = torch.optim.SGD(params, lr=1e-3,
                                momentum=0.8, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=5,
                                                   gamma=1)

    # let's train it for 10 epochs
    num_epochs = 20

    for epoch in range(num_epochs):
        print('/////////////// Epoch N°%d ///////////////////'%(epoch))
        # train for one epoch, printing every 10 iterations
        print('--------- TRAIN ----------')
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        print('---------- EVAL ----------')
        evaluate(model, data_loader_test, device=device)
        torch.save(model,'/content/drive/Shared drives/Projet ROBOT/segmentation/MASK_RCNN.pth')

    print("That's it!")

if __name__ == "__main__":
    main()


/////////////// Epoch N°0 ///////////////////
--------- TRAIN ----------
Epoch: [0]  [ 0/17]  eta: 0:01:21  lr: 0.000063  loss: 5.4160 (5.4160)  loss_classifier: 2.3595 (2.3595)  loss_box_reg: 0.6519 (0.6519)  loss_mask: 2.3341 (2.3341)  loss_objectness: 0.0537 (0.0537)  loss_rpn_box_reg: 0.0168 (0.0168)  time: 4.7831  data: 3.9197  max mem: 10446
Epoch: [0]  [10/17]  eta: 0:00:30  lr: 0.000688  loss: 4.4966 (4.2781)  loss_classifier: 1.9246 (1.8122)  loss_box_reg: 0.6274 (0.6270)  loss_mask: 1.7891 (1.7222)  loss_objectness: 0.0879 (0.0951)  loss_rpn_box_reg: 0.0219 (0.0215)  time: 4.3631  data: 3.4993  max mem: 10446
Epoch: [0]  [16/17]  eta: 0:00:04  lr: 0.001000  loss: 3.1540 (3.6015)  loss_classifier: 1.2680 (1.5008)  loss_box_reg: 0.6274 (0.6238)  loss_mask: 1.1276 (1.3818)  loss_objectness: 0.0605 (0.0729)  loss_rpn_box_reg: 0.0220 (0.0222)  time: 4.4161  data: 3.5816  max mem: 10446
Epoch: [0] Total time: 0:01:15 (4.4162 s / it)
---------- EVAL ----------
creating index...
inde